In [1]:
import tensorflow as tf
import cv2
import sys
import time 
import random
import numpy as np
from collections import deque
from matplotlib import pyplot as plt
import pygame
from ple import PLE

/home/cn1lab005/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


In [2]:
from ple.games.catcher import Catcher
#game = Catcher()
game = Catcher(width=256, height=256)
game.screen = pygame.display.set_mode(game.getScreenDims(), 0, 32)

In [3]:
GAME  = "CATCHER"
ACTIONS = 3 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 500. # timesteps to observe before training
EXPLORE = 500. # frames over which to anneal epsilon
FINAL_EPSILON =  0.05 # final value of epsilon
INITIAL_EPSILON = 1 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

In [4]:
epsilon = 1e-9
iter_routing = 2
train_freq = 10
tf.reset_default_graph()

In [5]:
def squash(vector):
    vec_squared_norm = reduce_sum(tf.square(vector), -2, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)
    vec_squashed = scalar_factor * vector  # element-wise
    return(vec_squashed)
def reduce_sum(input_tensor, axis=None, keepdims=False):
    return tf.reduce_sum(input_tensor, axis=axis, keepdims=keepdims)
def softmax(logits, axis=None):
    return tf.nn.softmax(logits, axis=axis)
def routing(input, b_IJ):
    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    W = tf.get_variable('Weight', shape=(1, 32, 50, 5, 1), dtype=tf.float32,
                        initializer=tf.random_normal_initializer(stddev=0.01))
    biases = tf.get_variable('bias', shape=(1, 1, 10, 5, 1))
    # A better solution is using element-wise multiply, reduce_sum and reshape
    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as
    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and
    # reshape to [a, c]
    input = tf.tile(input, [1, 1, 50, 1, 1])
    #assert input.get_shape() == [cfg.batch_size, 1024, 160, 8, 1]

    u_hat = reduce_sum(W * input, axis=3, keepdims=True)
    u_hat = tf.reshape(u_hat, shape=[-1, 32, 10, 5, 1])
    #assert u_hat.get_shape() == [cfg.batch_size, 1024, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(iter_routing):
        with tf.variable_scope('iter_' + str(r_iter)):
            # line 4:
            # => [batch_size, 1024, 10, 1, 1]
            c_IJ = softmax(b_IJ, axis=2)

            # At last iteration, use `u_hat` in order to receive gradients from the following graph
            if r_iter == iter_routing - 1:
                # line 5:
                # weighting u_hat with c_IJ, element-wise in the last two dims
                # => [batch_size, 1024, 10, 16, 1]
                s_J = tf.multiply(c_IJ, u_hat)
                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                #assert s_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]

                # line 6:
                # squash using Eq.1,
                v_J = squash(s_J)
                #assert v_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]
            elif r_iter < iter_routing - 1:  # Inner iterations, do not apply backpropagation
                s_J = tf.multiply(c_IJ, u_hat_stopped)
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                v_J = squash(s_J)

                # line 7:
                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1024, 10, 16, 1]
                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the
                # batch_size dim, resulting in [1, 1024, 10, 1, 1]
                v_J_tiled = tf.tile(v_J, [1, 32, 1, 1, 1])
                u_produce_v = reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keepdims=True)
                #assert u_produce_v.get_shape() == [cfg.batch_size, 1024, 10, 1, 1]

                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
                b_IJ += u_produce_v
    return(v_J)

In [6]:
def createNetwork():
    s= tf.placeholder("float", [None, 28, 28, 4])
    coeff = tf.placeholder(tf.float32, shape=(None, 32, 10, 1, 1))
    ####################### New Network COnfiguration #####################    
    w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
    w1 = tf.get_variable('w1',[6, 6, 4, 16],initializer=w_initializer)
    b1 = tf.get_variable('b1',[16],initializer=b_initializer)

    l1 = tf.nn.conv2d(s, w1, strides=[1, 2, 2, 1], padding="VALID")

    conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))
    #print("stu",conv1)
    conv1 = tf.reshape(conv1,[-1,12,12,16])
    #print(conv1)

    capsules1 = tf.contrib.layers.conv2d(conv1, 10, kernel_size=6, stride=2, padding="VALID",
                    activation_fn = tf.nn.relu,
                    weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                    biases_initializer=tf.constant_initializer(0))
    #print(capsules1,"jhg")

    capsules = tf.reshape(capsules1, (-1, 32, 5, 1)) #Reshape to(batch_szie, 1152, 8, 1)
    #print(capsules)
    capsules = squash(capsules)
    #print(capsules)

    input_caps2 = tf.reshape(capsules, shape=(-1, 32, 1, capsules.shape[-2].value, 1))
    #print(capsules)

    caps2 = routing(input_caps2, coeff)
    #print(caps2)

    vector_j = tf.reshape(caps2, shape=(-1, 50))
    #print(vector_j)
    q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)
    #print(q_eval)
    readout = q_eval
    return s, coeff, readout

In [7]:
def trainNetwork(s, coeff, readout, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices = 1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

    ## open up a game state to communicate with emulator
    # game_state = game.GameState()
    p =  PLE(game,display_screen=True)
    actions = p.getActionSet()
    p.init()
    reward = p.act(p.NOOP)
    
    # store the previous observations in replay memory
    D = deque()
    
    # get the first state by doing nothing and preprocess the image to 80x80x4
    
    action_index =  np.random.randint(0, 3)
    reward = p.act(actions[action_index])
    if(reward != 0):
        if(reward < -1):
            r_0 = -1
        else:
            r_0 = reward
        terminal = 1
        if(np.max(readout_t) > qmax):
            qmax = np.max(readout_t) 
        print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
        qmax = -10000
    else:
        terminal = 0
        r_0 = reward
            
    x_t = p.getScreenRGB()
    x_t = cv2.cvtColor(cv2.resize(x_t, (28, 28)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    sess.run(tf.global_variables_initializer())
    # saving and loading networks
    saver = tf.train.Saver()
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")
    b_IJ1 = np.zeros((1, 32, 10, 1, 1)).astype(np.float32) # batch_size=1
    b_IJ2 = np.zeros((BATCH, 32, 10, 1, 1)).astype(np.float32) # batch_size=BATCH
    epsilon = INITIAL_EPSILON
    t = 0
    score = 0
    qmax = -100000
    tick = time.time()
    while True:
        # choose an action epsilon greedily
        readout_t = readout.eval(feed_dict = {s:s_t.reshape((1,28,28,4)), coeff:b_IJ1})
        
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if random.random() <= epsilon or t <= OBSERVE:
            action_index = random.randrange(ACTIONS)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(readout_t)
            a_t[action_index] = 1

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        reward = p.act(actions[action_index])
        if(reward != 0):
            if(reward < -1):
                r_t = -1
            else:
                r_t = reward
            terminal = 1
            if(np.max(readout_t) > qmax):
                qmax = np.max(readout_t) 
            print("time_step", t,"/SCORE", score,"/rt", r_t,"/ Q_MAX %e" % qmax)
            qmax = -10000
        else:
            terminal = 0
            r_t = reward
        # run the selected action and observe next state and reward
        x_t1_colored = p.getScreenRGB()
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored, (28, 28)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (28, 28, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()
        
        # only train if done observing
        if t > OBSERVE and t%train_freq==0:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s:s_j1_batch, coeff:b_IJ2 })
            #readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})
            loss = cost.eval(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})

        # update the old values
        s_t = s_t1
        t += 1
        
        # save progress every 10000 iterations
        #if t % 10000 == 0:
        #    saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        score += reward
        if p.game_over():
            p.reset_game()
            #print("time_step:",t,"score:",score)
            score = 0
        #vars = tf.trainable_variables()
        #vars_vals = sess.run(vars)
        #print(vars_vals)
        #return vars_vals
        

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
s, coeff, readout = createNetwork()
vars_vals = trainNetwork(s, coeff, readout, sess)

INFO:tensorflow:Restoring parameters from saved_networks/CATCHER-dqn-40000
Successfully loaded: saved_networks/CATCHER-dqn-40000
time_step 41 /SCORE 0 /rt 1.0 / Q_MAX -2.079601e-02
time_step 77 /SCORE 1.0 /rt -1.0 / Q_MAX -2.246365e+00
time_step 118 /SCORE 0.0 /rt -1.0 / Q_MAX -2.800973e+00
time_step 164 /SCORE -1.0 /rt -1 / Q_MAX -2.651603e+00
time_step 199 /SCORE 0.0 /rt 1.0 / Q_MAX -1.549535e-01
time_step 235 /SCORE 1.0 /rt 1.0 / Q_MAX 9.501515e-01
time_step 274 /SCORE 2.0 /rt -1.0 / Q_MAX -2.353255e+00
time_step 310 /SCORE 1.0 /rt -1.0 / Q_MAX -2.157486e+00
time_step 353 /SCORE 0.0 /rt 1.0 / Q_MAX -7.733086e-01
time_step 389 /SCORE 1.0 /rt -1 / Q_MAX -2.134864e+00
time_step 421 /SCORE 0.0 /rt 1.0 / Q_MAX 9.334372e-01
time_step 464 /SCORE 1.0 /rt 1.0 / Q_MAX -6.126940e-01
time_step 500 /SCORE 2.0 /rt -1.0 / Q_MAX -2.540221e+00
time_step 546 /SCORE 1.0 /rt -1.0 / Q_MAX -8.309834e-01
time_step 582 /SCORE 0.0 /rt -1 / Q_MAX -2.274508e+00
time_step 624 /SCORE 0.0 /rt 1.0 / Q_MAX -6.4610

time_step 5568 /SCORE 10.0 /rt 1.0 / Q_MAX 9.388728e-01
time_step 5601 /SCORE 11.0 /rt 1.0 / Q_MAX 1.178648e+00
time_step 5634 /SCORE 12.0 /rt 1.0 / Q_MAX 7.336274e-01
time_step 5673 /SCORE 13.0 /rt 1.0 / Q_MAX -3.465714e-01
time_step 5709 /SCORE 14.0 /rt 1.0 / Q_MAX 9.843141e-01
time_step 5748 /SCORE 15.0 /rt 1.0 / Q_MAX 9.611741e-01
time_step 5787 /SCORE 16.0 /rt 1.0 / Q_MAX 1.280342e+00
time_step 5823 /SCORE 17.0 /rt 1.0 / Q_MAX 1.374329e+00
time_step 5859 /SCORE 18.0 /rt 1.0 / Q_MAX 1.263838e+00
time_step 5902 /SCORE 19.0 /rt 1.0 / Q_MAX 9.431718e-01
time_step 5943 /SCORE 20.0 /rt 1.0 / Q_MAX 5.805961e-01
time_step 5976 /SCORE 21.0 /rt 1.0 / Q_MAX 1.099847e+00
time_step 6010 /SCORE 22.0 /rt 1.0 / Q_MAX 1.249258e+00
time_step 6054 /SCORE 23.0 /rt -1.0 / Q_MAX -7.663337e-03
time_step 6087 /SCORE 22.0 /rt 1.0 / Q_MAX 1.095211e+00
time_step 6124 /SCORE 23.0 /rt -1 / Q_MAX 8.045769e-01
time_step 6159 /SCORE 0.0 /rt -1.0 / Q_MAX -4.550495e-01
time_step 6202 /SCORE -1.0 /rt 1.0 / Q_MAX 9.

time_step 11120 /SCORE 7.0 /rt 1.0 / Q_MAX 1.156063e+00
time_step 11156 /SCORE 8.0 /rt 1.0 / Q_MAX 7.852602e-01
time_step 11194 /SCORE 9.0 /rt 1.0 / Q_MAX 1.106652e+00
time_step 11228 /SCORE 10.0 /rt -1.0 / Q_MAX 5.244451e-01
time_step 11267 /SCORE 9.0 /rt 1.0 / Q_MAX 6.722974e-01
time_step 11310 /SCORE 10.0 /rt 1.0 / Q_MAX 5.948424e-01
time_step 11341 /SCORE 11.0 /rt 1.0 / Q_MAX 1.052795e+00
time_step 11379 /SCORE 12.0 /rt 1.0 / Q_MAX 1.112022e+00
time_step 11410 /SCORE 13.0 /rt 1.0 / Q_MAX 4.654559e-01
time_step 11441 /SCORE 14.0 /rt 1.0 / Q_MAX 1.168672e+00
time_step 11479 /SCORE 15.0 /rt 1.0 / Q_MAX 9.282122e-01
time_step 11513 /SCORE 16.0 /rt 1.0 / Q_MAX 1.111273e+00
time_step 11546 /SCORE 17.0 /rt 1.0 / Q_MAX 1.113861e-01
time_step 11588 /SCORE 18.0 /rt -1.0 / Q_MAX 3.625799e-01
time_step 11619 /SCORE 17.0 /rt 1.0 / Q_MAX 1.029151e+00
time_step 11650 /SCORE 18.0 /rt 1.0 / Q_MAX 1.101393e+00
time_step 11686 /SCORE 19.0 /rt 1.0 / Q_MAX 9.759569e-01
time_step 11722 /SCORE 20.0 /rt -